In [2]:
import openpyxl as xls
from scipy.stats import ttest_ind
from scipy.stats.contingency import chi2_contingency
import numpy as np
import re

In [3]:
import pandas as pd

In [4]:
bdc_data = pd.read_excel('230915 1600 분당차 데이터셋 김주연.xlsx', sheet_name='8672', header=11)

In [156]:
bdc_data

,year,Hospital,TYPE1,환자번호_2,환자번호_2.1,가명처리,분만일자_4,분만주_5,분만일_6,EDC_7,...,두번째 출생체중,두번째 합병증,두번째 COMMENT,세번째 분만년도,세번째 분만임신주수,세번째 분만형태,세번째 성별,세번째 출생체중,세번째 합병증,세번째 COMMENT
0,2015,분당차,정상,11427012,11427012,강*진,2015-06-03,40,6,2015-05-28,...,NaN,NaN,현재 임신중,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,분당차,정상,11418357,11418357,강*영,2015-06-09,39,0,2015-06-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,분당차,정상,11334675,11334675,강*사,2015-04-23,37,5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,분당차,정상,11545481,11545481,강*화,2015-09-03,39,0,2015-09-10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,분당차,정상,11584851,11584851,강*아,2015-05-28,38,1,2015-06-10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8667,2022,분당차,분만,11479420,11479420,황*혜,2022-09-06,38,1,2022-09-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8668,2022,분당차,분만,12161785,12161785,황*아,2022-03-22,39,0,2022-03-29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8669,2022,분당차,분만,12137290,12137290,황*신,2022-01-23,40,1,2022-01-22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8670,2022,분당차,분만,12126800,12126800,후*모토마이코,2022-02-03,39,1,2022-02-09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [135]:
print(bdc_data['08.21\n임당군/정상군 분류'].unique())
cases = bdc_data.loc[bdc_data['08.21\n임당군/정상군 분류']=='GDM']
conts = bdc_data.loc[bdc_data['08.21\n임당군/정상군 분류']=='정상군']
len(cases)+len(conts), len(cases), len(conts)

['정상군' 'GDM' '제외군(검사안함)' '제외군(overtDM)' '제외군(GDM-> overDM)'
 '제외군(정상군 -> overtDM)' '제외군(검사안함) -> overtDM']


(8106, 840, 7266)

In [34]:
def __find_name_of_col_bundanglogicnum__(x, tar_str):
    res = re.findall('[^0-9]_'+tar_str+'(?:$|[^0-9])',x)
    if len(res)<1:
        return False
    elif (res[0][2:]==tar_str)|(res[0][2:-1]==tar_str):
        return True
    else:
        return False
def find_name_of_col_bundanglogicnum(logicnum):
    i_col = np.where([__find_name_of_col_bundanglogicnum__(x,logicnum) for x in bdc_data.columns])
    name_col = bdc_data.columns[i_col]
    return name_col
# print(find_name_of_col_bundanglogicnum('16_1'))
k=1.96
def analy_for_conti_values(cases_0, conts_0, ):
    cases_ = cases_0.dropna()
    conts_ = conts_0.dropna()
    print('Missings:', len(cases_0)+len(conts_0)-len(cases_)-len(conts_))
    case_m, case_sd, cont_m, cont_sd = np.mean(cases_),np.std(cases_),np.mean(conts_),np.std(conts_)
    print('  {:.2f} ({:.2f},{:.2f})'.format(case_m, case_m-k*case_sd/(len(cases_)**0.5), case_m+k*case_sd/(len(cases_)**0.5)), end=' ')
    print('  {:.2f} ({:.2f},{:.2f})'.format(cont_m, cont_m-k*cont_sd/(len(cases_)**0.5), cont_m+k*cont_sd/(len(cases_)**0.5)), end=' ')
    print('  {:.4f}'.format(ttest_ind(list(cases_), list(conts_))[1]))
def analy_for_cate_values(cases_0, conts_0, ):
    cases_ = cases_0.dropna()
    conts_ = conts_0.dropna()
    print('Missings:', len(cases_0)+len(conts_0)-len(cases_)-len(conts_))
    case0, case1, cont0, cont1 = sum(cases_==0), sum(cases_==1), sum(conts_==0), sum(conts_==1)
    obs = np.zeros(shape=(2,2))
    obs[0,0] = case0; obs[0,1] = cont0;
    obs[1,0] = case1; obs[1,1] = cont1;
    obs = obs.astype(np.int64); 
    sum_case, sum_cont = sum(obs[:,0])/100.0, sum(obs[:,1])/100.0
    print('{:d}({:.1f}) {:d}({:.1f})'.format(obs[0,0], obs[0,0]/sum_case, obs[0,1], obs[0,1]/sum_cont))
    print('{:d}({:.1f}) {:d}({:.1f})'.format(obs[1,0], obs[1,0]/sum_case, obs[1,1], obs[1,1]/sum_cont))
    print("  {:.3f}".format(chi2_contingency(obs)[1]))

In [14]:
col_names = find_name_of_col_bundanglogicnum('9')
print(col_names); selected_col = col_names[0]
cases_, conts_ = cases[selected_col], conts[selected_col]
print('************************************')
analy_for_conti_values(cases_, conts_)

Index(['만나이분만일기준_9'], dtype='object')
************************************
Missings: 0
  35.86 (35.60,36.13)   34.28 (34.01,34.56)   0.0000


In [35]:
col_names = find_name_of_col_bundanglogicnum('16_1')
print(col_names); selected_col = col_names[-1]
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if x=='자연임신':
        return 0
    return 1
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

Index(['이번임신방법_16_1', '이번임신방법_16_1.1', '이번임신방법_16_1.2'], dtype='object')
[nan '시술_체외수정(IVF)' '시술_인공수정(IUI)' '약_배란유도제사용' '자연임신' '체외수정(IVF)'
 '인공수정(IUI)' '배란유도제사용']
************************************
Missings: 4557
163(38.0) 1332(42.7)
266(62.0) 1788(57.3)
  0.073


In [37]:
col_names = find_name_of_col_bundanglogicnum('16_1') #엑셀 표에는 16_2를 참조하라고 했으나 16_1을 기준으로 하는 게 맞다고 판단하였음.
print(col_names)
selected_col = col_names[-1]
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if 'IVF' in x:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

Index(['이번임신방법_16_1', '이번임신방법_16_1.1', '이번임신방법_16_1.2'], dtype='object')
************************************
Missings: 4557
193(45.0) 1614(51.7)
236(55.0) 1506(48.3)
  0.010


In [39]:
col_names = find_name_of_col_bundanglogicnum('35') 
print(col_names)
selected_col = col_names[1]
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if '유'==x:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

Index(['흡연력_35', '흡연력_35.1'], dtype='object')
['무' nan '유']
************************************
Missings: 12
829(98.9) 7218(99.5)
9(1.1) 38(0.5)
  0.081


In [40]:
col_names = find_name_of_col_bundanglogicnum('36')
print(col_names); selected_col = col_names[1]
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if '유'==x:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

Index(['음주력_36', '음주력_36.1'], dtype='object')
['무' nan '유']
************************************
Missings: 12
837(99.9) 7229(99.6)
1(0.1) 27(0.4)
  0.385


In [109]:
col_names = find_name_of_col_bundanglogicnum('52')
print(col_names)
selected_col = col_names[0]
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if '유'==x:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

Index(['고혈압_52'], dtype='object')
[nan '무' '유']
************************************
Missings: 6542
192(97.0) 1349(98.8)
6(3.0) 17(1.2)
  0.102


In [119]:
selected_col = 'para 정보 가져오기'
print(selected_col)
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    elif (x=='0(G1)'):
        return 0
    elif type(x)==str:
        return 1
    elif x > 0:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
cases_para_indexes = cases__[cases__==1].index
conts_para_indexes = conts__[conts__==1].index
print('************************************')
analy_for_cate_values(cases__, conts__)

para 정보 가져오기
[1 0 2 3 4 '0->1' 5 '0->2' '0(G1)' 8]
************************************
Missings: 0
534(63.6) 4542(62.5)
306(36.4) 2724(37.5)
  0.573


In [44]:
selected_col = '8.21\n자연조산 재분류'
print(selected_col)
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if x=='자연조산아님':
        return -1
    if x=='자연조산':
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
cases__ = cases__.loc[cases_para_indexes]
conts__ = conts__.loc[conts_para_indexes]
print('************************************')
print(sum(cases__==-1)+sum(conts__==-1))
analy_for_cate_values(cases__, conts__)

8.21
자연조산 재분류
['만삭분만' '만삭분만_자연진통' '자연조산' '자연조산아님']
************************************
71
Missings: 0
265(89.2) 2461(92.4)
32(10.8) 201(7.6)
  0.065


In [125]:
selected_col = 'CH 유뮤 \n(반영 후_김)'
print(selected_col)
cases_, conts_ = cases[selected_col], conts[selected_col]
print(cases_.unique(), conts_.unique())
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if x in ['CH없음','CH 없음']:
        return 0
    return 1
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
cases__ = cases__.loc[cases_para_indexes]
conts__ = conts__.loc[conts_para_indexes]
print('************************************')
analy_for_cate_values(cases__, conts__)

CH 유뮤 
(반영 후_김)
['CH없음' 'CH 상병' 'CH(과거력)'] ['CH없음' 'CH(과거력)' 'CH 상병' 'CH 없음' 'CH']
************************************
Missings: 0
289(94.4) 2685(98.6)
17(5.6) 39(1.4)
  0.000


In [114]:
col_names = find_name_of_col_bundanglogicnum('40')
print(col_names)
selected_col = col_names[-1]
cases_, conts_ = cases[selected_col], conts[selected_col]
print(cases_.unique(), conts_.unique())
def tempfunc_(x):
    if pd.isna(x):
        return 0
    elif '유' in x:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
cases__ = cases__.loc[cases_para_indexes]
conts__ = conts__.loc[conts_para_indexes]
print('************************************')
analy_for_cate_values(cases__, conts__)

Index(['임신성당뇨_40', '임신성당뇨_40.1'], dtype='object')
[nan '유' '유(출산력변수)'] [nan '유' '유(출산력변수)']
************************************
Missings: 0
214(69.9) 2599(95.4)
92(30.1) 125(4.6)
  0.000


In [47]:
col_names = find_name_of_col_bundanglogicnum('424')
print(col_names)
selected_col = col_names[-1]
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
cases__ = pd.Series(0, index=cases_.index);
conts__ = pd.Series(0, index=conts_.index)
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    elif '유' in x:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

Index(['가족력당뇨_424', '가족력당뇨_424.1', '가족력당뇨_424.2', '가족력당뇨_424.3'], dtype='object')
['무' '유' nan]
************************************
Missings: 3
581(69.2) 6064(83.5)
258(30.8) 1200(16.5)
  0.000


In [48]:
col_names = find_name_of_col_bundanglogicnum('426')
print(col_names)
selected_col = col_names[-1]
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    elif '유' in x:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

Index(['가족력고혈압_426', '가족력고혈압_426.1', '가족력고혈압_426.2'], dtype='object')
['무' '유' nan]
************************************
Missings: 3
592(70.6) 5215(71.8)
247(29.4) 2049(28.2)
  0.478


In [49]:
col_names = find_name_of_col_bundanglogicnum('90')
print(col_names); selected_col = col_names[0]
# print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if ((x<15) or (x>50)):
        return np.nan
    return np.float32(x)
print('************************************')
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
analy_for_conti_values(cases__, conts__)

Index(['BMI_90', 'BMI_90.1'], dtype='object')
************************************
Missings: 6469
  24.03 (23.49,24.56)   22.16 (21.68,22.64)   0.0000


In [136]:
col_names = find_name_of_col_bundanglogicnum('90')
print(col_names); selected_col = col_names[0]
# print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if ((x<15) or (x>50)):
        return np.nan
    if x>=30:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

Index(['BMI_90', 'BMI_90.1'], dtype='object')
************************************
Missings: 6469
192(91.0) 1366(95.8)
19(9.0) 60(4.2)
  0.004


In [137]:
col_names = find_name_of_col_bundanglogicnum('88')
print(col_names)
selected_col = col_names[1]
# print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
print('************************************')
analy_for_conti_values(cases_, conts_)

Index(['혈압수축기_88', '혈압수축기_88.1'], dtype='object')
************************************
Missings: 5835
  123.23 (121.37,125.09)   117.15 (115.51,118.78)   0.0000


In [138]:
col_names = find_name_of_col_bundanglogicnum('89')
print(col_names)
selected_col = col_names[1]
#print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
print('************************************')
analy_for_conti_values(cases_, conts_)

Index(['혈압이완기_89', '혈압이완기_89.1'], dtype='object')
************************************
Missings: 5836
  73.44 (72.06,74.83)   68.99 (67.69,70.30)   0.0000


In [139]:
col_names = find_name_of_col_bundanglogicnum('88')
print(col_names)
selected_col = col_names[1]
cases_, conts_ = cases[selected_col], conts[selected_col]
col_names = find_name_of_col_bundanglogicnum('89')
print(col_names)
selected_col = col_names[1]
cases2_, conts2_ = cases[selected_col], conts[selected_col]
print('************************************')
analy_for_conti_values((cases_+cases2_*2)/3, (conts_+conts2_*2)/3)

Index(['혈압수축기_88', '혈압수축기_88.1'], dtype='object')
Index(['혈압이완기_89', '혈압이완기_89.1'], dtype='object')
************************************
Missings: 5836
  90.04 (88.60,91.48)   85.04 (83.73,86.35)   0.0000


In [151]:
selected_col = 'BMI11_94'
print(selected_col)
# print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
# print(cases_.unique(), conts_.unique())
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if (x==' ') or (x=='  '):
        return np.nan
    x = np.float32(x)
    if ((x<15) or (x>50)):
        return np.nan
    return x
print('************************************')
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
analy_for_conti_values(cases__, conts__)

BMI11_94
************************************
Missings: 5186
  24.86 (24.36,25.36)   22.51 (22.10,22.92)   0.0000


In [55]:
selected_col = 'BMI11_94'
print(selected_col)
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if (x==' ') or (x=='  '):
        return np.nan
    x = np.float32(x)
    if ((x<15) or (x>50)):
        return np.nan
    if x>=30:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

BMI11_94
************************************
Missings: 5186
243(85.9) 2531(96.0)
40(14.1) 106(4.0)
  0.000


In [56]:
selected_col = '혈압수축기11_91'
print(selected_col)
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if (x==' ') or (x=='  '):
        return np.nan
    return np.float32(x)
print('************************************')
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
analy_for_conti_values(cases__, conts__)

혈압수축기11_91
************************************
Missings: 2714
  117.89 (116.82,118.97)   113.18 (112.16,114.21)   0.0000


In [57]:
selected_col = '혈압이완기11_92'
print(selected_col)
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if (x==' ') or (x=='  '):
        return np.nan
    return np.float32(x)
print('************************************')
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
analy_for_conti_values(cases__, conts__)

혈압이완기11_92
************************************
Missings: 2713
  72.59 (71.68,73.49)   69.43 (68.42,70.44)   0.0000


In [58]:
selected_col = '혈압이완기11_92'
print(selected_col)
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if (x==' ') or (x=='  '):
        return np.nan
    return np.float32(x)
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
selected_col = '혈압수축기11_91'
print(selected_col)
cases_, conts_ = cases[selected_col], conts[selected_col]
cases2__ = cases_.apply(lambda x: tempfunc_(x))
conts2__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_conti_values((cases__*2+cases2__)/3, (conts__*2+conts2__)/3)

혈압이완기11_92
혈압수축기11_91
************************************
Missings: 2714
  87.69 (86.79,88.59)   84.01 (83.12,84.90)   0.0000


In [144]:
selected_col = 'BMI16_98'
print(selected_col)
# print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if (x==' ') or (x=='  '):
        return np.nan
    x = np.float32(x)
    if ((x<15) or (x>=50)):
        return np.nan
    return x
print('************************************')
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
analy_for_conti_values(cases__, conts__)

BMI16_98
************************************
Missings: 5456
  25.33 (24.87,25.78)   23.15 (22.75,23.54)   0.0000


In [60]:
selected_col = 'BMI16_98'
print(selected_col)
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if (x==' ') or (x=='  '):
        return np.nan
    x = np.float32(x)
    if ((x<15) or (x>50)):
        return np.nan
    if x>=30:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

BMI16_98
************************************
Missings: 5456
265(86.3) 2227(95.0)
42(13.7) 116(5.0)
  0.000


In [61]:
selected_col = '혈압수축기16_95'
print(selected_col)
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if (x==' ') or (x=='  '):
        return np.nan
    return np.float32(x)
print('************************************')
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
analy_for_conti_values(cases__, conts__)

혈압수축기16_95
************************************
Missings: 1992
  116.96 (115.97,117.95)   112.04 (111.14,112.94)   0.0000


In [62]:
selected_col = '혈압이완기16_96'
print(selected_col)
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if (x==' ') or (x=='  '):
        return np.nan
    return np.float32(x)
print('************************************')
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
analy_for_conti_values(cases__, conts__)

혈압이완기16_96
************************************
Missings: 1994
  71.38 (69.35,73.40)   67.11 (65.93,68.28)   0.0000


In [63]:
selected_col = '혈압이완기16_96'
print(selected_col)
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if (x==' ') or (x=='  '):
        return np.nan
    return np.float32(x)
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
selected_col = '혈압수축기16_95'
print(selected_col)
cases_, conts_ = cases[selected_col], conts[selected_col]
cases2__ = cases_.apply(lambda x: tempfunc_(x))
conts2__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_conti_values((cases__*2+cases2__)/3, (conts__*2+conts2__)/3)

혈압이완기16_96
혈압수축기16_95
************************************
Missings: 1994
  86.57 (85.11,88.03)   82.08 (81.14,83.03)   0.0000


In [145]:
selected_col = 'BMI20_101'
print(selected_col)
# print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if (x==' ') or (x=='  '):
        return np.nan
    x = np.float32(x)
    if ((x<15) or (x>50)):
        return np.nan
    return x
print('************************************')
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
analy_for_conti_values(cases__, conts__)

BMI20_101
************************************
Missings: 5148
  26.12 (25.70,26.54)   24.03 (23.66,24.39)   0.0000


In [65]:
selected_col = 'BMI20_101'
print(selected_col)
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if (x==' ') or (x=='  '):
        return np.nan
    x = np.float32(x)
    if ((x<15) or (x>50)):
        return np.nan
    if x>=30:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

BMI20_101
************************************
Missings: 5148
295(83.3) 2430(93.3)
59(16.7) 174(6.7)
  0.000


In [66]:
selected_col = '혈압수축기20_99_1'
print(selected_col)
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if (x==' ') or (x=='  '):
        return np.nan
    return np.float32(x)
print('************************************')
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
analy_for_conti_values(cases__, conts__)

혈압수축기20_99_1
************************************
Missings: 1631
  116.50 (115.58,117.43)   112.11 (111.25,112.97)   0.0000


In [67]:
selected_col = '혈압이완기20_99_2'
print(selected_col)
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if (x==' ') or (x=='  '):
        return np.nan
    return np.float32(x)
print('************************************')
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
analy_for_conti_values(cases__, conts__)

혈압이완기20_99_2
************************************
Missings: 1631
  70.75 (68.94,72.57)   66.58 (65.67,67.48)   0.0000


In [68]:
selected_col = '혈압이완기20_99_2'
print(selected_col)
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if (x==' ') or (x=='  '):
        return np.nan
    return np.float32(x)
cases__ = cases_.apply(lambda x: tempfunc_(x))
conts__ = conts_.apply(lambda x: tempfunc_(x))
selected_col = '혈압수축기20_99_1'
print(selected_col)
cases_, conts_ = cases[selected_col], conts[selected_col]
cases2__ = cases_.apply(lambda x: tempfunc_(x))
conts2__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_conti_values((cases__*2+cases2__)/3, (conts__*2+conts2__)/3)

혈압이완기20_99_2
혈압수축기20_99_1
************************************
Missings: 1632
  86.00 (84.68,87.32)   81.75 (80.97,82.53)   0.0000


In [156]:
selected_col = '함+산\nPAPP-A(MoM)'
print(selected_col)
a = np.where(bdc_data.columns==selected_col)[0][0]
for i in range(1,20):
    if '차주' in bdc_data.columns[a-i]:
        print(bdc_data.columns[a-i]);week_i = a-i
        break
print(bdc_data[selected_col].unique())
print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif x == ' ':
        return False
    elif (float(x)<=22)&(float(x)>=0):
        return True
    return False
cases_, conts_ = cases[cases.iloc[:,week_i].apply(lambda x: func_for_week(x))], conts[conts.iloc[:,week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col], conts_[selected_col]
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

함+산
PAPP-A(MoM)
INTEGRATED1차주_122
[0.504   nan 1.263 ... 2.557 2.184 2.628]
['11' nan '12' '-65' '13' '10' '14' '-116' '-45' '-82' '-238' '-273' '15'
 '-93' '-134' '-172' '-178' '-145' 11 12 13 10 -117 ' ' -46 -77 -38 '-52']
************************************
5248
Missings: 40
  1.11 (1.05,1.16)   1.20 (1.15,1.26)   0.0011


In [157]:
selected_col = '함+산\nAFP MoM'
print(selected_col)
a = np.where(bdc_data.columns==selected_col)[0][0]
for i in range(1,20):
    if '차주' in bdc_data.columns[a-i]:
        print(bdc_data.columns[a-i]);week_i = a-i
        break
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[cases.iloc[:,week_i].apply(lambda x: func_for_week(x))], conts[conts.iloc[:,week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col], conts_[selected_col]
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

함+산
AFP MoM
INTEGRAGTED2차주_146
[0.806   nan 0.913 ... 1.506 1.193 2.467]
************************************
5401
Missings: 8
  1.10 (1.07,1.14)   1.07 (1.03,1.10)   0.0629


In [71]:
selected_col = '함+산\nTotalhCGB MoM'
print(selected_col)
a = np.where(bdc_data.columns==selected_col)[0][0]
for i in range(1,20):
    if '차주' in bdc_data.columns[a-i]:
        print(bdc_data.columns[a-i]);week_i = a-i
        break
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[cases.iloc[:,week_i].apply(lambda x: func_for_week(x))], conts[conts.iloc[:,week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col], conts_[selected_col]
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

함+산
TotalhCGB MoM
INTEGRAGTED2차주_146
[0.878   nan 0.787 ... 0.294 3.119 3.134]
************************************
5401
Missings: 8
  1.21 (1.15,1.27)   1.18 (1.12,1.23)   0.2587


In [72]:
selected_col = '함+산\nUE3 MoM'
print(selected_col)
a = np.where(bdc_data.columns==selected_col)[0][0]
for i in range(1,20):
    if '차주' in bdc_data.columns[a-i]:
        print(bdc_data.columns[a-i]);week_i = a-i
        break
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[cases.iloc[:,week_i].apply(lambda x: func_for_week(x))], conts[conts.iloc[:,week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col], conts_[selected_col]
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

함+산
UE3 MoM
INTEGRAGTED2차주_146
[0.666   nan 0.839 ... 1.959 1.456 1.378]
************************************
5401
Missings: 8
  0.99 (0.96,1.01)   1.03 (0.91,1.14)   0.4748


In [73]:
selected_col = '함+산\nINHIBIN MoM'
print(selected_col)
a = np.where(bdc_data.columns==selected_col)[0][0]
for i in range(1,20):
    if '차주' in bdc_data.columns[a-i]:
        print(bdc_data.columns[a-i]);week_i = a-i
        break
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[cases.iloc[:,week_i].apply(lambda x: func_for_week(x))], conts[conts.iloc[:,week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col], conts_[selected_col]
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

함+산
INHIBIN MoM
INTEGRAGTED2차주_146
[0.788   nan 1.042 ... 1.637 0.521 2.018]
************************************
5401
Missings: 8
  1.09 (1.04,1.13)   1.07 (1.02,1.12)   0.4745


In [158]:
col_names = find_name_of_col_bundanglogicnum('118_1')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns==selected_col)[0][0]
for i in range(1,20):
    if '주_계산' in bdc_data.columns[a+i]:
        print(bdc_data.columns[a+i]);week_i = a+i
        break
print(bdc_data[selected_col].unique())
print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    if x=='Error':
        return False
    if (float(x)<=22)&(float(x)>0):
        return True
    return False
cases_, conts_ = cases[cases.iloc[:,week_i].apply(lambda x: func_for_week(x))], conts[conts.iloc[:,week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col], conts_[selected_col]
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)
week_i

Index(['WBC_118_1'], dtype='object')
주_계산.8
[10.77 10.53  6.67 ... 12.74 13.3  18.76]
[11. 35. 10.  7. 37.  9. 34. 16. 12. 36. 13. 23. 38.  6. 26. 29. 33. 19.
 22. 32. 14. 25. 39. 27.  8. 17. 15. 18. nan 20. 28. 31.  3. 24. 30. 21.
 40.  5.  4.  0. -1. -4.  2. -2.  1. -5.]
************************************
3726
Missings: 0
  9.07 (8.85,9.29)   8.33 (8.13,8.53)   0.0000


329

In [75]:
col_names = find_name_of_col_bundanglogicnum('118_3')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns==selected_col)[0][0]
for i in range(1,20):
    if '주_계산' in bdc_data.columns[a+i]:
        print(bdc_data.columns[a+i]);week_i = a+i
        break
# print(bdc_data[selected_col].unique())
# print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x)<=22)&(float(x)>0):
        return True
    return False
cases_, conts_ = cases[cases.iloc[:,week_i].apply(lambda x: func_for_week(x))], conts[conts.iloc[:,week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col], conts_[selected_col]
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['HGB_118_3'], dtype='object')
주_계산.8
************************************
3726
Missings: 0
  12.62 (12.51,12.73)   12.47 (12.37,12.57)   0.0074


In [76]:
col_names = find_name_of_col_bundanglogicnum('118_4')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a + i]:
        print(bdc_data.columns[a + i]);week_i = a + i
        break
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col], conts_[selected_col]
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['HCT_118_4'], dtype='object')
주_계산.8
************************************
3726
Missings: 0
  37.17 (36.84,37.50)   36.80 (36.50,37.09)   0.0208


In [77]:
col_names = find_name_of_col_bundanglogicnum('118_9')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a + i]:
        print(bdc_data.columns[a + i]);week_i = a + i
        break
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col], conts_[selected_col]
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['PLT_118_9'], dtype='object')
주_계산.8
************************************
3726
Missings: 0
  265.90 (259.87,271.93)   247.28 (241.74,252.83)   0.0000


In [78]:
col_names = find_name_of_col_bundanglogicnum('118_30')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a - i]:
        print(bdc_data.columns[a - i]);week_i = a - i
        break
print(bdc_data[selected_col].unique())
print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if x is np.nan:
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['TSH_118_30', 'TSH_118_30.1', 'TSH_118_30.2'], dtype='object')
주_계산.19
['0.842' nan '0.582' ... 1.243 2.072 0.302]
[11. nan 10.  7. -1.  9. 12. 23.  6. 17. 34. 19. 15. 16.  8. 13. 25.  3.
 22. 26. 18. 30. 14. 20. 36. 38. 27. 21. 29.  0. 28. 35. 24.  4. 37.  1.
 31. 39.  5. 33.  2. 32. 40. -8.]
************************************
4475
Missings: 0
  1.38 (1.26,1.49)   1.30 (1.16,1.44)   0.2810


In [79]:
selected_col = 'FT4_118_31.2'
print(selected_col)
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a - i]:
        print(bdc_data.columns[a - i]);week_i = a - i
        break
# print(bdc_data[selected_col].unique())
# print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<','').replace('>',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

FT4_118_31.2
주_계산.20
************************************
3805
Missings: 0
  1.21 (1.19,1.24)   1.23 (1.20,1.26)   0.2832


In [80]:
selected_col = 'ANTITPO_118_32.2'
print(selected_col)
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a - i]:
        print(bdc_data.columns[a - i]);week_i = a - i
        break
# print(bdc_data[selected_col].unique())
# print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<','').replace('>',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

ANTITPO_118_32.2
주_계산.20
************************************
3805
Missings: 1350
  20.08 (13.79,26.38)   21.53 (14.23,28.83)   0.7096


In [81]:
col_names = find_name_of_col_bundanglogicnum('118_33')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a - i]:
        print(bdc_data.columns[a - i]);week_i = a - i
        break
# print(bdc_data[selected_col].unique())
# print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['BUN_118_33', 'BUN_118_33.1', 'BUN_118_33.2'], dtype='object')
주_계산.21
************************************
2741
Missings: 0
  7.92 (7.67,8.17)   7.96 (7.71,8.21)   0.7986


In [82]:
col_names = find_name_of_col_bundanglogicnum('118_34')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a - i]:
        print(bdc_data.columns[a - i]);week_i = a - i
        break
# print(bdc_data[selected_col].unique())
# print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['CREATININE_118_34', 'CREATININE_118_34.1', 'CREATININE_118_34.2'], dtype='object')
주_계산.22
************************************
2819
Missings: 0
  0.50 (0.49,0.51)   0.51 (0.50,0.52)   0.3517


In [83]:
col_names = find_name_of_col_bundanglogicnum('118_35')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a - i]:
        print(bdc_data.columns[a - i]);week_i = a - i
        break
# print(bdc_data[selected_col].unique())
# print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['GLUCOSE_118_35', 'GLUCOSE_118_35.1', 'GLUCOSE_118_35.2'], dtype='object')
주_계산.23
************************************
2287
Missings: 0
  100.92 (97.60,104.24)   90.44 (88.77,92.11)   0.0000


In [84]:
col_names = find_name_of_col_bundanglogicnum('118_36')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a - i]:
        print(bdc_data.columns[a - i]);week_i = a - i
        break
# print(bdc_data[selected_col].unique())
# print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['HGBA1C_118_36', 'HGBA1C_118_36.1', 'HGBA1C_118_36.2',
       'HGBA1C_118_36.3'],
      dtype='object')
주_계산.25
************************************
676
Missings: 0
  5.51 (5.40,5.63)   5.18 (5.12,5.25)   0.0000


In [85]:
col_names = find_name_of_col_bundanglogicnum('118_37')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a - i]:
        print(bdc_data.columns[a - i]);week_i = a - i
        break
#print(bdc_data[selected_col].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['25OHVitaminD_118_37', '25OHVitaminD_118_37.1', '25OHVitaminD_118_37.2',
       '25OHVitaminD_118_37.3'],
      dtype='object')
주_계산.26
************************************
3379
Missings: 0
  15.62 (14.60,16.65)   14.76 (13.83,15.69)   0.0885


In [86]:
col_names = find_name_of_col_bundanglogicnum('118_38')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a - i]:
        print(bdc_data.columns[a - i]);week_i = a - i
        break
# print(bdc_data[selected_col].unique())
# print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['HDLCHOLESTEROL_118_38', 'HDLCHOLESTEROL_118_38.1',
       'HDLCHOLESTEROL_118_38.2', 'HDLCHOLESTEROL_118_38.3'],
      dtype='object')
주_계산.28
************************************
780
Missings: 0
  65.40 (62.59,68.21)   70.03 (66.74,73.32)   0.0089


In [87]:
col_names = find_name_of_col_bundanglogicnum('118_39')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a - i]:
        print(bdc_data.columns[a - i]);week_i = a - i
        break
# print(bdc_data[selected_col].unique())
# print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['TRIGLYCERIDE_118_39', 'TRIGLYCERIDE_118_39.1',
       'TRIGLYCERIDE_118_39.2'],
      dtype='object')
주_계산.29
************************************
1147
Missings: 0
  173.55 (158.29,188.81)   131.96 (120.44,143.48)   0.0000


In [88]:
col_names = find_name_of_col_bundanglogicnum('118_40')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a - i]:
        print(bdc_data.columns[a - i]);week_i = a - i
        break
# print(bdc_data[selected_col].unique())
# print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['T.Protein_118_40', 'T.Protein_118_40.1', 'T.Protein_118_40.2'], dtype='object')
주_계산.30
************************************
2490
Missings: 0
  6.84 (6.79,6.90)   6.83 (6.78,6.88)   0.4880


In [89]:
col_names = find_name_of_col_bundanglogicnum('118_41')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a - i]:
        print(bdc_data.columns[a -i]);week_i = a - i
        break
# print(bdc_data[selected_col].unique())
# print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['ALBUMIN_118_41', 'ALBUMIN_118_41.1', 'ALBUMIN_118_41.2'], dtype='object')
주_계산.31
************************************
2562
Missings: 0
  4.13 (4.09,4.16)   4.13 (4.09,4.16)   0.9402


In [90]:
col_names = find_name_of_col_bundanglogicnum('118_42')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a - i]:
        print(bdc_data.columns[a - i]);week_i = a - i
        break
# print(bdc_data[selected_col].unique())
# print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['T.Bilirubin_118_42', 'T.Bilirubin_118_42.1', 'T.Bilirubin_118_42.2'], dtype='object')
주_계산.32
************************************
2635
Missings: 0
  0.34 (0.29,0.40)   0.35 (0.33,0.37)   0.6865


In [91]:
col_names = find_name_of_col_bundanglogicnum('118_43')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a - i]:
        print(bdc_data.columns[a - i]);week_i = a - i
        break
# print(bdc_data[selected_col].unique())
# print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['ASGOT_118_43', 'ASGOT_118_43.1', 'ASGOT_118_43.2'], dtype='object')
주_계산.33
************************************
4172
Missings: 0
  17.88 (16.93,18.84)   17.00 (16.21,17.79)   0.0445


In [92]:
col_names = find_name_of_col_bundanglogicnum('118_44')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a - i]:
        print(bdc_data.columns[a - i]);week_i = a - i
        break
# print(bdc_data[selected_col].unique())
# print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['ALTGPT_118_44', 'ALTGPT_118_44.1', 'ALTGPT_118_44.2'], dtype='object')
주_계산.34
************************************
4174
Missings: 0
  18.04 (16.59,19.50)   15.86 (14.43,17.29)   0.0049


In [93]:
col_names = find_name_of_col_bundanglogicnum('118_45')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a - i]:
        print(bdc_data.columns[a - i]);week_i = a - i
        break
# print(bdc_data[selected_col].unique())
# print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['ALP_118_45', 'ALP_118_45.1', 'ALP_118_45.2'], dtype='object')
주_계산.35
************************************
2470
Missings: 0
  68.55 (63.98,73.12)   64.59 (60.54,68.63)   0.0764


In [94]:
col_names = find_name_of_col_bundanglogicnum('118_46')
print(col_names)
selected_col = col_names[-1]
a = np.where(bdc_data.columns == selected_col)[0][0]
for i in range(1, 20):
    if '주_계산' in bdc_data.columns[a - i]:
        print(bdc_data.columns[a - i]);week_i = a - i
        break
# print(bdc_data[selected_col].unique())
# print(bdc_data.iloc[:,week_i].unique())
def func_for_week(x):
    if pd.isna(x):
        return False
    elif (float(x) <= 22) & (float(x) > 0):
        return True
    return False
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases[cases.iloc[:, week_i].apply(lambda x: func_for_week(x))], conts[
    conts.iloc[:, week_i].apply(lambda x: func_for_week(x))]
cases_, conts_ = cases_[selected_col].apply(lambda x: tempfunc(x)), conts_[selected_col].apply(lambda x: tempfunc(x))
print('************************************')
print(len(cases_)+len(conts_))
analy_for_conti_values(cases_, conts_)

Index(['TOTALCHOLESTEROL_118_46', 'TOTALCHOLESTEROL_118_46.1',
       'TOTALCHOLESTEROL_118_46.2'],
      dtype='object')
주_계산.36
************************************
2641
Missings: 0
  198.04 (194.09,201.99)   190.08 (186.04,194.11)   0.0003


In [95]:
selected_col = '08.21\n임당군/정상군 분류'
print(selected_col)
#print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
print(cases_.unique(), conts_.unique())
print('************************************')
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if x=='GDM':
        return 1
    if x=='정상군':
        return 0
    return -1
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
print('제외군', sum(cases__==-1)+sum(cases__==-1))
analy_for_cate_values(cases__, conts__)

08.21
임당군/정상군 분류
['GDM'] ['정상군']
************************************
************************************
제외군 0
Missings: 0
0(0.0) 7266(100.0)
840(100.0) 0(0.0)
  0.000


In [96]:
col_names = find_name_of_col_bundanglogicnum('5')
print(col_names)
selected_col = col_names[-1]
# print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases_.apply(lambda x: tempfunc(x)), conts_.apply(lambda x: tempfunc(x))
print('************************************')
analy_for_conti_values(cases_, conts_)

Index(['분만주_5'], dtype='object')
************************************
Missings: 0
  37.77 (37.66,37.89)   38.17 (38.07,38.28)   0.0000


In [97]:
col_names = find_name_of_col_bundanglogicnum('5')
print(col_names)
selected_col = col_names[-1]
#print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
print(cases_.unique(), conts_.unique())
print('************************************')
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    elif x<=37:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

Index(['분만주_5'], dtype='object')
[40 39 38 37 36 25 35 33 31 27 32 41 34 30 22] [40 39 37 38 36 33 34 27 35 41 32 30 31 29 28 26 25 42]
************************************
************************************
Missings: 0
570(67.9) 5659(77.9)
270(32.1) 1607(22.1)
  0.000


In [98]:
col_names = find_name_of_col_bundanglogicnum('334')
print(col_names)
selected_col = col_names[-1]
# print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif type(x)==str:
        return float(x.replace('<',''))
    return float(x)
cases_, conts_ = cases_.apply(lambda x: tempfunc(x)), conts_.apply(lambda x: tempfunc(x))
print('************************************')
analy_for_conti_values(cases_, conts_)

Index(['WTGM_334'], dtype='object')
************************************
Missings: 0
  3087.07 (3053.74,3120.39)   3144.77 (3114.95,3174.60)   0.0004


In [99]:
col_names = find_name_of_col_bundanglogicnum('334')
print(col_names)
selected_col = col_names[-1]
#print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
print('************************************')
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    elif x>=4000:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

Index(['WTGM_334'], dtype='object')
************************************
************************************
Missings: 0
818(97.4) 7147(98.4)
22(2.6) 119(1.6)
  0.055


In [100]:
col_names = find_name_of_col_bundanglogicnum('334')
print(col_names)
selected_col = col_names[-1]
#print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
#print(cases_.unique(), conts_.unique())
print('************************************')
def tempfunc_(x):
    if pd.isna(x):
        return False
    elif x<=2500:
        return True
    return False
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

Index(['WTGM_334'], dtype='object')
************************************
************************************
Missings: 0
758(90.2) 6829(94.0)
82(9.8) 437(6.0)
  0.000


In [101]:
selected_col = 'fenton curve\n(weight %)_674'
print(selected_col); print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
#print(cases_.unique(), conts_.unique())
print('************************************')
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if x=='성별 X':
        return np.nan
    elif float(x)>=0.9:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

fenton curve
(weight %)_674
[0.16111869103726667 0.6739711942982225 0.5949395466936785 ...
 0.38021437092800264 0.31399242962411256 0.6492741333970126]
************************************
************************************
Missings: 1
805(95.9) 7060(97.2)
34(4.1) 206(2.8)
  0.063


In [102]:
selected_col = 'fenton curve\n(weight %)_674'
print(selected_col)
#print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    if x=='성별 X':
        return np.nan
    elif float(x)<=0.1:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

fenton curve
(weight %)_674
************************************
Missings: 1
769(91.7) 6753(92.9)
70(8.3) 513(7.1)
  0.197


In [103]:
selected_col = '1분 7점 미만 (1)'
print(selected_col)
#print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
print(cases_.unique(), conts_.unique())
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    elif x==1:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

1분 7점 미만 (1)
[ 0. nan  1.] [ 0. nan  1.]
************************************
Missings: 1768
606(96.0) 5516(96.7)
25(4.0) 191(3.3)
  0.489


In [104]:
selected_col = '5분7점 미만(1)'
print(selected_col)
#print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
#print(cases_.unique(), conts_.unique())
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    elif x==1:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

5분7점 미만(1)
************************************
Missings: 1202
684(99.1) 6154(99.0)
6(0.9) 60(1.0)
  0.968


In [105]:
col_names = find_name_of_col_bundanglogicnum('338')
print(col_names); selected_col = col_names[-1]
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if x=='Y':
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

Index(['NICU입원여부_338'], dtype='object')
[nan 'Y']
************************************
Missings: 0
700(83.3) 6266(86.2)
140(16.7) 1000(13.8)
  0.025


In [106]:
col_names = find_name_of_col_bundanglogicnum('349')
print(col_names)
selected_col = col_names[-1]
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc(x):
    if pd.isna(x):
        return np.nan
    elif x=='Error':
        return np.nan
    return np.float32(x)
cases_, conts_ = cases_.apply(lambda x: tempfunc(x)), conts_.apply(lambda x: tempfunc(x))
print('************************************')
analy_for_conti_values(cases_, conts_)

Index(['BST_349'], dtype='object')
[nan '82' '66' '74' '62' '72' '114' '60' '63' '58' '94' '65' '70' '29'
 '61' '81' '95' '47' '49' '57' '87' '77' '64' '89' '35' '48' '75' '52'
 '79' '68' '54' '45' '55' '69' '46' '97' '73' '44' '102' '83' '34' '42'
 '86' '67' '21' '80' '59' '56' '41' '23' '76' '98' '39' '107' '84' '92'
 '38' '71' '85' '101' '53' '78' '90' '24' '120' '88' '91' '50' '103' '51'
 '43' '100' '93' '113' '36' '104' '96' '111' '99' '112' '28' '109' '106'
 '25' 'Error' '40' '30' '31' '115' '33' '32' 73 64 60 58 67 76 78 81 57 63
 70 41 72 75 52 85 66 68 65 51 71 61 79 62 80 53 86 54 77 82 47 44 69 38
 36 29 104 43 84 50 74 45 28 59 48 88 55 35 46 56 42 92 49 34 40 89 91 83
 93 31 37 102 26 90 125 '37' '105' 30 33 25 27 127 39 103 94 24]
************************************
Missings: 5882
  61.20 (60.50,61.90)   66.19 (65.13,67.24)   0.0000


In [107]:
col_names = find_name_of_col_bundanglogicnum('349')
print(col_names)
selected_col = col_names[-1]
# print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
def tempfunc_(x):
    if pd.isna(x):
        return np.nan
    elif x=='Error':
        return np.nan
    elif float(x)<=50:
        return 1
    return 0
cases__ = cases_.apply(lambda x: tempfunc_(x)) 
conts__ = conts_.apply(lambda x: tempfunc_(x))
print('************************************')
analy_for_cate_values(cases__, conts__)

Index(['BST_349'], dtype='object')
************************************
Missings: 5882
703(85.8) 1196(85.1)
116(14.2) 209(14.9)
  0.692


In [108]:
col_names = find_name_of_col_bundanglogicnum('351')
print(col_names)
selected_col = col_names[-1]
print(bdc_data[selected_col].unique())
cases_, conts_ = cases[selected_col], conts[selected_col]
print('모두 nan value라 분석 불가.')

Index(['SEIZURE_351'], dtype='object')
[nan]
모두 nan value라 분석 불가.
